In [71]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [72]:
def date_str_parser(dt_str):
    return dt.strptime(dt_str, '%d-%m-%Y')

In [73]:
df_orig = pd.read_csv('FIFA_2018_Statistics_orig.csv', parse_dates=[0], date_parser=date_str_parser)


# Example of preprocessing copied from: https://www.kaggle.com/mathvv/seaborn-tutorial-and-fifa2018-mom-prediction 

In [74]:
df_ex = df_orig.copy()

In [75]:
dropset = ['PSO','Goals in PSO', 'Own goals','Own goal Time']
df_ex.drop(dropset,axis=1,inplace=True)

In [76]:
df_ex.replace(('Yes', 'No'), (1, 0), inplace=True)
df_ex['1st Goal'] = df_ex['1st Goal'].fillna(0)
#change first goal to categorical
df_ex['1st Goal'].loc[(df_ex['1st Goal'] >= 1) & (df_ex['1st Goal'] < 15)] = 1
df_ex['1st Goal'].loc[(df_ex['1st Goal'] >= 15) & (df_ex['1st Goal'] < 30)] = 2
df_ex['1st Goal'].loc[(df_ex['1st Goal'] >= 30) & (df_ex['1st Goal'] < 45)] = 3
df_ex['1st Goal'].loc[(df_ex['1st Goal'] >= 45) & (df_ex['1st Goal'] < 60)] = 4
df_ex['1st Goal'].loc[(df_ex['1st Goal'] >= 60) & (df_ex['1st Goal'] < 75)] = 5
df_ex['1st Goal'].loc[df_ex['1st Goal'] >= 75] = 6

In [78]:
categories = ['Goal Scored','1st Goal','Ball Possession %','Attempts','On-Target','Blocked','Corners','Offsides','Free Kicks','Saves','Pass Accuracy %','Passes','Distance Covered (Kms)','Fouls Committed']
n = len(categories)

ex_a = df_ex[categories].values.reshape((-1, n))
ex_y = df_ex.loc[::2, 'Man of the Match'].values
ex_m = ex_a.shape[0] / 2
ex_x = np.zeros((ex_m, n))

# convert statistics into relative measures w.r.t. opponent's values
ex_x[:,0:2] = ex_a[::2,0:2] - ex_a[1::2,0:2] # On a goal difference
ex_x[:,2:] = ex_a[::2,2:] / (ex_a[::2,2:] + ex_a[1::2,2:]) * 100 # Proportion
np.nan_to_num(ex_x,copy=False);

/home/astrophysics/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in divide
  # This is added back by InteractiveShellApp.init_path()


In [79]:
np.savetxt("FIFA_2018_Statistics_data1.csv", np.concatenate([ex_x, ex_y.reshape(-1,1)], axis=1), delimiter=",")

In [80]:
#separate group and knockout stage data
df_group = df_ex[df_ex.Round == 'Group Stage']
df_group.drop('Round',axis=1,inplace=True)
df_knockout = df_ex[df_ex.Round != 'Group Stage']
df_knockout.drop('Round',axis=1,inplace=True)

In [82]:
#look at group and knockout data separately
group_a = df_group[categories].values.reshape((-1, n))
group_y = df_group.loc[::2, 'Man of the Match'].values
group_m = group_a.shape[0] / 2
group_x = np.zeros((group_m, n))

# convert statistics into relative measures w.r.t. opponent's values
group_x[:,0:2] = group_a[::2,0:2] - group_a[1::2,0:2] # On a goal difference
group_x[:,2:] = group_a[::2,2:] / (group_a[::2,2:] + group_a[1::2,2:]) * 100 # Proportion
np.nan_to_num(group_x,copy=False)

knockout_a = df_knockout[categories].values.reshape((-1, n))
knockout_y = df_knockout.loc[::2, 'Man of the Match'].values
knockout_m = knockout_a.shape[0] / 2
knockout_x = np.zeros((knockout_m, n))

knockout_x[:,0:2] = knockout_a[::2,0:2] - knockout_a[1::2,0:2] 
knockout_x[:,2:] = knockout_a[::2,2:] / (knockout_a[::2,2:] + knockout_a[1::2,2:]) * 100 
np.nan_to_num(knockout_x,copy=False);


/home/astrophysics/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
/home/astrophysics/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in divide


# Own attempt at preprocessing data

In [ ]:
df_num = df_orig.copy()

In [4]:
#get team values, get unique values (by appyling set), then arrange alphabetically in a list
countries = sorted(set(df_orig['Team'].values))
n_countries = len(countries)
#create integer index for countries, and map to alphabetic representation
countries_I = np.arange(n_countries)
countries_dict = dict(zip(countries, countries_I))
# {'Argentina': 0,
#  'Australia': 1,
#  'Belgium': 2,
#  'Brazil': 3,
#  'Colombia': 4,
#  'Costa Rica': 5,
#  'Croatia': 6,
#  'Denmark': 7,
#  'Egypt': 8,
#  'England': 9,
#  'France': 10,
#  'Germany': 11,
#  'Iceland': 12,
#  'Iran': 13,
#  'Japan': 14,
#  'Korea Republic': 15,
#  'Mexico': 16,
#  'Morocco': 17,
#  'Nigeria': 18,
#  'Panama': 19,
#  'Peru': 20,
#  'Poland': 21,
#  'Portugal': 22,
#  'Russia': 23,
#  'Saudi Arabia': 24,
#  'Senegal': 25,
#  'Serbia': 26,
#  'Spain': 27,
#  'Sweden': 28,
#  'Switzerland': 29,
#  'Tunisia': 30,
#  'Uruguay': 31}
#it appears that .loc is slower here (used on either side of assignment...
df_num['Team'] = df_orig['Team'].apply(lambda x: countries_dict[x])
#df_num['Opponent'] = df_orig['Opponent'].apply(lambda x: countries_dict[x])

In [5]:
#replace nan values
nan_fill = 0
df_num.fillna(nan_fill, inplace=True)
#replace yes/no values with 1/0
df_num.replace(('Yes', 'No'), (1, 0), inplace=True)
#replace Rounds (Group Stage, Round of 16,  Quarter Finals, Semi-Finals, 3rd Place, Final) with 0/1/2/3/4/5
df_num.replace(('Group Stage', 'Round of 16', 'Quarter Finals', 'Semi- Finals', '3rd Place', 'Final'), (1, 2, 3, 4, 5, 6), inplace=True)
#convert dates to integer. first day of tournament = 1, all other dates are counts of calendar days from first calendar day + 1. 
df_num['Date'] = df_num['Date'].dt.strftime("%Y%m%d").astype(int) - df_num['Date'].dt.strftime("%Y%m%d").astype(int).min() + 1

In [6]:
df_team = df_num.loc[::2]
df_team.drop('Opponent', axis=1, inplace=True)
df_team.rename(columns = dict(zip(df_team.columns.values, df_team.columns.values + ' 1')), inplace=True)
df_team.rename(columns = {'Date 1': 'Date', 'Round 1': 'Round', 'Man of the Match 1': 'Man of the Match'}, inplace=True)
df_team.reset_index(inplace=True)

/home/astrophysics/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/astrophysics/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [7]:
df_opp = df_num.loc[1::2]
#n.b. don't need ball possession of second team as it is linear combination of that of first (100 - first ball possession)
df_opp.drop(['Opponent', 'Date', 'Ball Possession %', 'Round', 'Man of the Match'], axis=1, inplace=True)
df_opp.rename(columns = dict(zip(df_opp.columns.values, df_opp.columns.values + ' 2')), inplace=True)
df_opp.reset_index(inplace=True)

In [8]:
df_concat = pd.concat([df_team, df_opp], axis=1)
df_cols = df_concat.columns.tolist()
#get rid of second index
df_cols = df_cols[:27] + df_cols[28:]
#put date to back of columns. get rid of first index
df_cols = df_cols[2:] + [df_cols[1]]
#put round to back of columns
df_cols = df_cols[:20] + df_cols[21:] + [df_cols[20]]
#put man of the match to back of columns
df_cols = df_cols[:18] + df_cols[19:] + [df_cols[18]]
df_concat = df_concat.loc[:, df_cols]

In [9]:
df_concat.to_csv('FIFA_2018_Statistics_num.csv', index=False)

In [10]:
cols = df_concat.columns.values
num_cols = len(cols)
cols_ind = np.arange(num_cols)
cols_dict = dict(zip(cols, cols_ind))
#cols_dict
# {'1st Goal 1': 18,
#  '1st Goal 2': 40,
#  'Attempts 1': 3,
#  'Attempts 2': 25,
#  'Ball Possession % 1': 2,
#  'Blocked 1': 6,
#  'Blocked 2': 28,
#  'Corners 1': 7,
#  'Corners 2': 29,
#  'Date': 45,
#  'Distance Covered (Kms) 1': 13,
#  'Distance Covered (Kms) 2': 35,
#  'Fouls Committed 1': 14,
#  'Fouls Committed 2': 36,
#  'Free Kicks 1': 9,
#  'Free Kicks 2': 31,
#  'Goal Scored 1': 1,
#  'Goal Scored 2': 24,
#  'Goals in PSO 1': 20,
#  'Goals in PSO 2': 42,
#  'Man of the Match': 47,
#  'Off-Target 1': 5,
#  'Off-Target 2': 27,
#  'Offsides 1': 8,
#  'Offsides 2': 30,
#  'On-Target 1': 4,
#  'On-Target 2': 26,
#  'Own goal Time 1': 22,
#  'Own goal Time 2': 44,
#  'Own goals 1': 21,
#  'Own goals 2': 43,
#  'PSO 1': 19,
#  'PSO 2': 41,
#  'Pass Accuracy % 1': 11,
#  'Pass Accuracy % 2': 33,
#  'Passes 1': 12,
#  'Passes 2': 34,
#  'Red 1': 17,
#  'Red 2': 39,
#  'Round': 46,
#  'Saves 1': 10,
#  'Saves 2': 32,
#  'Team 1': 0,
#  'Team 2': 23,
#  'Yellow & Red 1': 16,
#  'Yellow & Red 2': 38,
#  'Yellow Card 1': 15,
#  'Yellow Card 2': 37}

In [70]:
#to start with, just use columns which I suspect are the most useful.
indices = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,24,25,26,27,28,29,30,31,32,33,34,35,36,47]
fifa_a = df_concat.iloc[:, indices].values
np.savetxt("FIFA_2018_Statistics_data2.csv", fifa_a, delimiter=",")